In [1]:
!pip install rdflib
!pip install requests


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import requests
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, SKOS, OWL, RDF, RDFS, VOID, XMLNS, XSD
from rdflib import Graph, URIRef, Literal
import json
g = Graph()
file_path = '/content/drive/MyDrive/IAOS/title_Ner.txt'
with open(file_path, "r") as file:
    text = file.read()
    paragraphs = text.split("\n")
    for paragraph in paragraphs:
      if "->" in paragraph:
        sujeto = paragraph.split("->")[0].strip()
        texto = paragraph.split('->')[1]
        objeto =  texto[texto.find("'word'"):]
        objeto = objeto[objeto.find("word") + len("word"): objeto.find(",")]
        objeto = objeto[objeto.find("':'") + len("word'"): objeto.find(",")]
        
        sujeto = sujeto.replace(" ", "_")
        g.add((
          URIRef("http://IAOS.com/resource/paper/"+sujeto),
          URIRef("http://IAOS.com/def/property#" + "acknowledges"),
          Literal(objeto, datatype=XSD.string)))
    

 
 

In [7]:

propiedad= "acknowledges"
file_path = '/content/drive/MyDrive/IAOS/resultadoOpenAlex.txt'
clase="author"
with open(file_path, "r") as file:
    text = file.read()
    paragraphs = text.split("\n")
    for paragraph in paragraphs:
      partes = paragraph.split("---")
      
      if(len(partes)==3):
        sujeto = partes[0]
        propiedad = partes[1]
        objeto = partes[2]
        sujeto = sujeto.replace(" ", "_")
        g.add((
            URIRef("http://IAOS.com/resource/"+clase+"/"+sujeto),
            URIRef("http://IAOS.com/def/property#" + propiedad),
            Literal(objeto, datatype=XSD.string)))
        if(clase=='author'):
          clase="paper"
        elif(clase=='paper'):
          clase='author'
      

 

In [8]:
topic_path = '/content/drive/MyDrive/IAOS/Title_topic_prob.txt' # Replace with the actual file path

# Read the lines and store the content in triples
title_prob_topic = []
with open(topic_path, 'r') as file:
    for line in file:
        line = line.strip()  # Remove leading/trailing whitespaces
        parts = line.split('---')  # Split the line by '---' separator
        if len(parts) == 3:  # Ensure the line contains three parts
            title = parts[0].strip()
            probability = float(parts[1].strip())
            topic = parts[2].strip()
            title_prob_topic.append((title, probability, topic))

# Print the triples

i=0
for triple in title_prob_topic:
    g.add((
      URIRef("http://IAOS.com/resource/paper/"+triple[0].replace(" ", "_")),
      URIRef("http://IAOS.com/def/property#"+"paper_belongs_to"),
      URIRef("http://IAOS.com/resource/belongs_to/"+str(i))))
    g.add((
      URIRef("http://IAOS.com/resource/belongs_to/"+str(i)),
      URIRef("http://IAOS.com/def/property#"+"similar_to_paper"),
      Literal(triple[1], datatype=XSD.float)))
    g.add((
      URIRef("http://IAOS.com/resource/belongs_to/"+str(i)),
      URIRef("http://IAOS.com/def/property#"+"belongs_to_topic"),
      Literal(triple[2], datatype=XSD.string)))
    i=i+1






In [9]:
topic_path = '/content/drive/MyDrive/IAOS/Topic_words.txt' # Replace with the actual file path

# Read the lines and store the content in triples
triples = []
with open(topic_path, 'r') as file:
    for line in file:
        line = line.strip()  # Remove leading/trailing whitespaces
        parts = line.split(':')  # Split the line by '---' separator
        if len(parts) == 2:  # Ensure the line contains three parts
            topic = parts[0].strip()
            words = parts[1].strip()
            palabras = words.split(' ')
            for palabra in palabras:
              triples.append((topic, palabra, "hasWord"))



# Add the triples to the graph
for triple in triples:
    g.add((
      URIRef("http://IAOS.com/resource/topic/"+triple[0].replace(" ", "_")),
      URIRef("http://IAOS.com/def/property#"+"hasWord"),
      Literal(triple[1], datatype=XSD.string)))
   

In [10]:
import json
wikidata_path = '/content/drive/MyDrive/IAOS/resultadoWikiData.txt' 
triples = []
with open(wikidata_path, 'r') as file:
    for line in file:
        line = line.strip()  # Remove leading/trailing whitespaces
        parts = line.split('---')  # Split the line by '/' separator
        if len(parts) == 2:  # Ensure the line contains three parts
          title = parts[0].strip()
          wikidata = parts[1].strip()
          wikidata = wikidata.replace("'", '"')
          json_dict = json.loads(wikidata)

          # Extract the variables and their values
          variables = json_dict['head']['vars']
          bindings = json_dict['results']['bindings']

          # Print the variables and their values
          for binding in bindings:
              for variable in variables:
                  value = binding.get(variable, {}).get('value', None)
                  relacion=""
                  objeto=""
                  if(variable=="doi"):
                    relacion= URIRef("http://IAOS.com/def/property#"+"hasDoi")
                    objeto = Literal(value, datatype=XSD.string)
                  
                  elif (variable=="cites"):
                      relacion= URIRef("http://IAOS.com/def/property#"+"hasCites")
                      objeto = Literal(value, datatype=XSD.string)
                  
                  elif(variable=="num_pag" or variable=="pages"):
                      relacion= URIRef("http://IAOS.com/def/property#"+"hasNumPages")
                      objeto = Literal(value, datatype=XSD.integer)
                  
                  elif(variable=="publication_date"):
                      relacion= URIRef("http://IAOS.com/def/property#"+"hasPublicationDate")
                      objeto = Literal(value, datatype=XSD.date)
                  elif(variable=="language"):
                      relacion= URIRef("http://IAOS.com/def/property#"+"hasLanguage")
                      objeto = Literal(value, datatype=XSD.string)
                  if(value is not None):
                      g.add((
                      URIRef("http://IAOS.com/resource/paper/"+title.replace(" ", "_")),
                      relacion,
                      objeto))


In [11]:

from rdflib import Namespace
onto = Namespace("http://IAOS.com/def/property#")
g.bind("onto", onto)

while True:
  try:
    knows_query = input("Enter some text: ")
    qres = g.query(knows_query)

    for row in qres:
        print(row)
  except Exception as e:
    print("Not a valid query")

Enter some text: uuw
Not a valid query


KeyboardInterrupt: ignored